Data cleaning:

Sets organization

In [ ]:
import pandas as pd
import os

path = "./../data/raw/"

rent = [file for file in os.listdir(path) if file.startswith("apartments_rent_pl")]
purchase = [file_2 for file_2 in os.listdir(path) if file_2.startswith("apartments_pl")]

print(rent, purchase)

whole_rent = pd.DataFrame()
whole_rent = whole_rent.to_csv("./../data/interim/1_whole_rent.csv", index=False)

whole_purchase = pd.DataFrame()
whole_purchase.to_csv("./../data/interim/2_whole_purchase.csv", index=False)

combined_data = []
combined_data2 = []

for file in rent:
    data = pd.read_csv(path + file)
    time = file[-6:-4] + "_" + file[-9:-7]
    print(time)
    print(file)
    data["Period"] = time
    combined_data.append(data)

result = pd.concat(combined_data, ignore_index=True)
result.to_csv("./../data/interim/1_whole_rent.csv", index=False)

for file_2 in purchase:
    data = pd.read_csv(path + file_2)
    time2 = file_2[-6:-4] + "_" + file_2[-9:-7]
    print(time2)
    data["Period"] = time2
    combined_data2.append(data)

result2 = pd.concat(combined_data2, ignore_index=True)
result2.to_csv("./../data/interim/2_whole_purchase.csv", index=False)

Data overview

In [ ]:
import pandas as pd

data=pd.read_csv("./../data/interim/1_whole_rent.csv")
print("Data of rent overview: ")
print("\n Shape od data", data.shape)
print("\n Lack of values: ")
print(data.isnull().sum())
print("\n Stats:")
print(data.describe(include='all'))

data=pd.read_csv("./../data/interim/2_whole_purchase.csv")
print("Data of sales overview: ")
print("\n Shape od data", data.shape)
print("\n Lack of values: ")
print(data.isnull().sum())


print("\n Stats:")
print(data.describe(include='all'))

1. For NaN in columns hasStorageRoom, hasSecurity, has Elevator, hasBalcony, and hasParkingSpace the NaN is filled with no
For NaN in columns with distances, the NaN is  filled with distance to the center
3. For Na in columns with condition, NaN is filled with "Low"
4. Floor is  filled with floor count for apartment

In [ ]:
import pandas as pd
import os
import numpy as np

fill_na_map = {'schoolDistance': "centreDistance", 'clinicDistance': "centreDistance", 'postOfficeDistance': "centreDistance",
               'kindergartenDistance': "centreDistance", 'restaurantDistance': "centreDistance", 'collegeDistance': "centreDistance",
               'pharmacyDistance': "centreDistance", "floor": "floorCount"}


path = "./../data/interim/"
files = os.listdir(path)
print(files)
print((pd.read_csv(path+files[0])).columns)

for file in files:
    data = pd.read_csv(path+file)
    # Empty cells for elevator, parking space, security, balcony,ect. is filled with no
    data.fillna({"hasParkingSpace": "no", 'hasBalcony': "no", 'hasElevator': "no", 'hasSecurity': "no",
                'hasStorageRoom': "no", "condition": "NoData", "buildingMaterial": "NoData", "type": "NoData"}, inplace=True)

    # Empty cells for distances from object is filled with distance to center
    for target, source in fill_na_map.items():
        data[target] = data[target].fillna(data[source])

    # Random year for year, but in rang of 1850(minial)-1930.
    nan_mask = data["buildYear"].isna()
    data.loc[nan_mask, "buildYear"] = np.random.randint(
        1850, 1930, size=nan_mask.sum())
    data = data.dropna(how='any')
    data.to_csv(path+file, index=False)
    

In [ ]:
import pandas as pd


data = pd.read_csv("./../data/interim/1_whole_rent.csv")
print("Data of rent overview: ")
print("\n Shape od data", data.shape)
print("\n Lack of values: ")
print(data.isnull().sum())

data = pd.read_csv("./../data/interim/2_whole_purchase.csv")
print("Data of sales overview: ")
print("\n Shape od data", data.shape)
print("\n Lack of values: ")
print(data.isnull().sum())

Wrocław, Warszawa, Lublin, Łódz, Radom data

In [ ]:
import pandas as pd
import os

path = "./../data/interim/"
files = os.listdir(path)
print(files)

for file in files:
    data = pd.read_csv(path+file)
    data = data[data["city"].isin(
        ["krakow", "warszawa", "radom", "lublin", "wroclaw"])]
    print(data["city"].unique())

    # Additional cleaning
    if "Unnamed: 0" in data.columns:
        data.drop(columns=["Unnamed: 0"], inplace=True)

    int_columns = ["rooms", "floor", "floorCount", 'bulidYear']
    for col in int_columns:
        if col in data.columns:
            data[col] = data[col].astype("Int64")

    binary_map = {"yes": True, "no": False, 'Yes': True, 'No': False}
    binary_columns = ['hasParkingSpace', 'hasBalcony',
                      'hasElevator', 'hasSecurity', 'hasStorageRoom']
    for col in binary_columns:
        if col in data.columns:
            data[col] = data[col].map(binary_map)

    if 'id' in data.columns:
        data['id'] = data['id'].astype(str)

    if 'buildYear' in data.columns:
        current_year = pd.Timestamp.now().year
        data['buildingAge'] = current_year - data['buildYear']
        data.loc[(data['buildingAge'] < 0) | (data['buildingAge'] > 300)]

    data.to_csv("./../data/processed/"+file[0]+"_analysis"+file[7:])

Data check

In [ ]:
import pandas as pd
import os

path = "./../data/processed/"
files = os.listdir(path)
print(files)

for file in files:
    data = pd.read_csv(path+file)
    print("\n============== Check for ", file, "===========")
    print("-------Missing values---------")
    print(data.isnull().sum())
    print('\n-----------Data Types------------')
    print(data.dtypes)
    print("\n----------Duplicate Rows---------")
    print(data.duplicated().sum())
    print("\n--------Summary Statistics-------")
    print(data.describe(include="all"))
    print("\n --------Unique values per column-----------")
    for col in data.columns:
        unique_v = data[col] = data[col].nunique()
        print(f"{col}:{unique_v} unique values")
    print("\n ------ Outliers Detection----")
    n_cols = data.select_dtypes(include=["number"]).columns
    for col in n_cols:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3-q1
        outliers = data[(data[col] < q1-1.5*iqr) | (data[col] > q3+1.5*iqr)]
        print(f"{col}:{len(outliers)} outliers")

    print(data.columns)